In [399]:
import numpy as np 
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import sys
from dotenv import load_dotenv
load_dotenv()

True

In [400]:
token = SpotifyClientCredentials(client_id= os.getenv('SPOTIFY_CLIENT_ID'), client_secret= os.getenv('SPOTIFY_SECRET'))
sp = spotipy.Spotify(client_credentials_manager= token)

In [424]:
# Fetch songs from a single playlist
def fetch_songs(playlist_id):
    playlist_features_list = ["artist", "album", "track_name", "track_id"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    results = sp.playlist_tracks(playlist_id)
    tracks_data = results['tracks']
    tracks = []

    # Collect every page of track data
    print('started reading tracks from playlist')
    while tracks_data['next']:
        tracks_data = sp.next(tracks_data)
        tracks.extend(tracks_data['items'])
    print('finished reading tracks')
    for track in tracks:
        playlist_features = {}
        
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["popularity"] = track["track"]["popularity"]

        # Concat
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    ids = playlist_df['track_id'].to_list()
    
    # Grab all the song analysis data
    analysis_features = ["acousticness", "liveness", "loudness", "speechiness", "tempo", 
                         "time_signature", "tempo", "energy", "instrumentalness", "time_signature", "id"]
    analysis_df = pd.DataFrame()
    analysis_data = {}
    print('started reading features')
    # for i in range(len(ids)):
        # TODO: Need to make less requests
        # song_analysis = sp.audio_features(ids[i])
        # for j in range(len(analysis_features)):
            # analysis_data[analysis_features[j]] = song_analysis[0][analysis_features[j]]
        #curr_anal_df = pd.DataFrame(analysis_data, index = [0])
        #analysis_df = pd.concat([analysis_df, curr_anal_df], ignore_index = True)
    i = 0
    print(len(playlist_features))
    while i <= len(ids):
        song_analysis = sp.audio_features(ids[i:i+99])
        print(song_analysis)
        for j in range(len(analysis_features)):
            for anal in song_analysis:
                analysis_data[analysis_features[j]] = anal[analysis_features[j]]
        print(analysis_data)
        i += 99
        print(i)
        curr_anal_df = pd.DataFrame(analysis_data, index = [0])
        analysis_df = pd.concat([analysis_df, curr_anal_df], ignore_index = True)
    print('finished reading features')

    # TODO: Remove
    tracks_df = pd.merge(left=playlist_df, right=analysis_df, left_on='track_id', right_on='id')

    return tracks_df 
    


In [425]:
fetch_songs('4rnleEAOdmFAbRcNCgZMpY?si=4c088aeade4b43e5')

started reading tracks from playlist


In [ ]:


#Function skeletons

def groupby_genre(songs_df):
    pass
def groupby_artist(songs_df):
    pass
def groupby_popularity(songs_df):
    pass
def groupby_audio_features(songs_df):
    pass